In [2]:
import cv2
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import math

from utils.line import *

In [ ]:
img_path = "/home/rp24/file/test_img/3_1.png"
lower_hsv = np.array([0, 0, 240])   # 直线色域 0 0 245
upper_hsv = np.array([180, 255, 255])


frame = cv2.imread(img_path)
height, width, _ = frame.shape

lines = get_lines(frame, lower_hsv, upper_hsv)
cal = get_center_and_angle_len(lines)

cal.shape


In [ ]:
plt.imshow(frame)

# Add lines on the frame
num_start = 0
num_end = num_start + 44
for idx in range(lines[num_start:num_end, ...].shape[0]):
    x0, y0, x1, y1 = lines[idx][0]
    center, angle = calculate_line_center_and_angle(x0, y0, x1, y1)
    print(f"center: {center}, angle: {angle:.1f}, {x0}, {y0} {x1}, {y1}")
    plt.plot([x0, x1], [y0, y1], color='red', linewidth=2)

# Show the plot
plt.show()

# capture

In [5]:
cap = cv2.VideoCapture(0)

output_dir = "/home/rp24/file/debug_output"

idx = 0
ret, FrameImage = cap.read()
print(ret)

False


[ WARN:0] global /tmp/pip-wheel-w_lllcn3/opencv-python_622962b8a47a42ed9a5e55c557dab0ed/opencv/modules/videoio/src/cap_v4l.cpp (893) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


# ref

In [ ]:
from utils.line import *
img_path = "/home/rp24/file/test_img/aw4.jpg"

lower_hsv = np.array([0, 0, 40])   # 直线色域 0 0 245
upper_hsv = np.array([180, 255, 255])

frame = cv2.imread(img_path)
frame = cv2.resize(frame,(600, 400),)
height, width, _ = frame.shape

lines = get_lines(frame, lower_hsv, upper_hsv)
cal = get_center_and_angle_len(lines)

cal.shape

In [ ]:
tt = cluster(cal)

In [ ]:
cal0 = cal[tt == 0]
cal1 = cal[tt == 1]
print(f"ca0: {cal0.shape}, ca1: {cal1.shape}")

In [ ]:
def get_key_lines(lines, std_threshold):
    cal = get_center_and_angle_len(lines)
    var = np.var(cal, axis=0)
    n_lines = 1 if max(var[0], var[1]) < std_threshold else 2
    clusters = cluster(cal, n_lines)
    if n_lines == 1:
        return n_lines, lines[0], cal[0]
    else:
        cal0 = cal[clusters == 0]
        cal1 = cal[clusters == 1]
        lines0 = lines[clusters == 0]
        lines1 = lines[clusters == 1]
        return n_lines, (lines0[0], lines1[0]), (cal0[0], cal1[0])

In [ ]:
a,b,c = get_key_lines(lines, 10)
print(a)
print(b)
print(c)

In [ ]:
def get_focus_area(frame, lines):
    pass

In [ ]:
import matplotlib.pyplot as plt
plt.figure()

img_path = "/home/rp24/file/test_img/aw3.jpg"
img = cv2.imread(img_path)
for cx, cy, angle in ca0:
    # cv2.circle(img, center=(cx, cy), color=(0, 0, 255), radius=20, thickness=2)
    cv2.circle(img, (cx, cy), (0, 0, 255), 20, 2)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
import cv2
img_path = "/home/rp24/file/test_img/aw3.jpg"
import matplotlib.pyplot as plt

# Load the image
img_path = "/home/rp24/file/test_img/aw3.jpg"
img = cv2.imread(img_path)

# Convert the image to different color spaces
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Display the images using matplotlib
plt.figure(figsize=(12, 6))

for i in range(3):
    plt.subplot(int(f"33{i*3+1}"))
    plt.imshow(img_rgb[..., i])
    plt.title("RGB")

for i in range(3):
    plt.subplot(int(f"33{i*3+2}"))
    plt.imshow(img_hsv[..., i])
    plt.title("HSV")

plt.subplot(int(f"133"))
plt.imshow(img_gray, cmap='gray')
plt.title("Grayscale")

plt.tight_layout()
plt.show()


# h5

In [1]:
import numpy as np
import h5py

In [2]:
# 1. 写入
f = h5py.File('./demo.h5', 'w')     # 打开文件，更推荐用with管理上下文

print(f.name)        # '/',  file相当于root
print(f.keys())      # <KeysViewHDF5 []>, 所有group的名称，相当于索引

/
<KeysViewHDF5 []>


In [3]:
# 直接赋值
A = np.random.rand(100,100)
f['numpy_A'] = A    # f[key]=value直接赋值，存储对应的group（numpy_A）和dataset(A)
for name in f:    # 等同于for name in f.keys()
    print(name)   # numpy_A

numpy_A


In [4]:
# 创建组
f.create_group('group_1')   # 和numpy_A同级
  # numpy_A  group_1print(f.keys())

<KeysViewHDF5 ['group_1', 'numpy_A']>


In [5]:
# 组内通过赋值的方式创建dataset
B = np.random.rand(200,200)
f['group_1']['numpy_B'] = B

In [14]:
print(f.keys())

<KeysViewHDF5 ['group_1', 'numpy_A']>


In [11]:
f['group_1'].create_dataset('mydataset12', (10, 10), dtype='f')

<HDF5 dataset "mydataset12": shape (10, 10), type "<f4">

In [15]:
# 通过/直接同时创建group和dataset
f.create_dataset('group_2/mydataset2', (20,20), dtype='f')
print(f.keys())    # group_1 group_2 numpy_A

<KeysViewHDF5 ['group_1', 'group_2', 'numpy_A']>


In [16]:
# # 创建属性
# f.attrs['name'] = 'Mia'   

# 关闭，存储
f.close()

In [17]:
# 2. 读取
f = h5py.File('./demo.h5', 'r')

print(f.keys())
print()
# 获取属性的key
print(f.attrs.keys())
print()
# 递归遍历
f.visit(lambda x:print(x))

<KeysViewHDF5 ['group_1', 'group_2', 'numpy_A']>

<KeysViewHDF5 []>

group_1
group_1/mydataset
group_1/mydataset1
group_1/mydataset12
group_1/numpy_B
group_2
group_2/mydataset2
numpy_A


In [19]:
data = f['group_1/mydataset1'][:]
data.shape

(10, 10)

In [13]:
data = f['group_2'][:]
data.shape

TypeError: Accessing a group is done with bytes or str, not <class 'slice'>

# 颜色增强

In [2]:
import cv2

hue_change = 5  # 色调改变值 步长

img_path = "/home/rp24/code/underwater_robot/file/debug/frames_saved_20240607181720.jpg"
img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # 打开文件


# 通过cv2.cvtColor把图像从BGR转换到HSV
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

turn_green_hsv = img_hsv.copy()
# 色调hue变化范围5——50
for i in range(1, 11):
    turn_green_hsv[:, :, 0] = (turn_green_hsv[:, :, 0] + hue_change * i) % 180
    turn_green_img = cv2.cvtColor(turn_green_hsv, cv2.COLOR_HSV2BGR)
    new_img_path = img_path.replace(".jpg", f"_color_change_{i}.jpg")
    cv2.imwrite(new_img_path, turn_green_img)


In [ ]:
import cv2
import numpy as np

# 加载图像
image = cv2.imread('image.png')

# 将图像从BGR转换为HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 定义黑色的HSV颜色范围
lower_black = np.array([0,0,0])
upper_black = np.array([180, 255, 30])

# 定义蓝色的HSV颜色范围
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])

# 创建黑色和蓝色的颜色掩码
mask_black = cv2.inRange(hsv, lower_black, upper_black)
mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

# 对原图像应用掩码得到只有黑色和蓝色的图像
result_black = cv2.bitwise_and(image, image, mask=mask_black)
result_blue = cv2.bitwise_and(image, image, mask=mask_blue)

# 显示结果
cv2.imshow('image', image)
cv2.imshow('black', result_black)
cv2.imshow('blue', result_blue)

# cv2.waitKey(0)
# cv2.destroyAllWindows()